In [1]:
import numpy as np
import pandas as pd
%load_ext google.colab.data_table

In [ ]:
df = pd.read_csv("drive/MyDrive/Datasets/attacks.csv", encoding="ISO-8859-1")
display(df.columns)
df2 = df.drop(columns=["Case Number","Year","Country","Area","Location","Name", "Sex ","href formula", "href", "Case Number.1", "Case Number.2", "Unnamed: 22","Unnamed: 23",'Investigator or Source', 'pdf'])
df3 = df2.rename(columns={"Species ": "Species"})
df3.shape

In [ ]:
# 1 What are the most dangerous types of sharks to humans? 

# Based on the cases where the Species of the shark was known, there are 3 or 4 sharks that have the highest ratio of deaths when attacking, the Tiger- White- , Bull-  and Blue Shark

# 2 Are children more likely to be attacked by sharks?

# Based on the cases where the age is given, around 1/3 of the cases involves a child. I would say no, adults are more likely to be attacked by sharks.

# 3 Are shark attacks where sharks were provoked more or less dangerous? 

# With provoked attacks, there were 19 with a deadly ending, and 548 non-fatal. With unprovoked attacks there were 1349 fatal attacks, 3417 non-fatal. Based on 
# this values you could say that unprovoked attacks are more dangeres.

# 4 Are certain activities more likely to result in a shark attack? 
# Surfing and Swimming and Fishing come on top of the most common activities when there was a Shark Attack. 


In [3]:
# Using the code from Questionarie A to do research about the different columns to see how much empty values

def print_separator(sep, num, msg):
  print("\n")
  print(sep * num)
  print(f"{msg}")
  print(sep * num)


def look_at_unique_values(column): 
  unique_values_cutoff = 160
  unique_values = column.unique()
  num_unique_values = len(unique_values)
  if num_unique_values == len(column):
    print(f"Each value in the column is unique (total: {num_unique_values})")
  elif num_unique_values < unique_values_cutoff:
    print(f"Less than {unique_values_cutoff} unique values:")
    # We may get an error when sorting
    try:
      sorted = np.sort(unique_values)
      print("Values are sorted")
      display(list(sorted))
    except:
      print("Could not sort values")
      display(list(unique_values))
  else:
    print(f"More than {unique_values_cutoff} unique values (total: {num_unique_values})")


def look_at_edges(df, column_name):
  # inner function
  def show_head_and_tail(values):
      num_items_to_slice = 10
      display(list(values)[:num_items_to_slice])
      display(list(values)[-num_items_to_slice:])

  column = df[column_name]
  unique_values = column.unique()
  try:
      sorted = np.sort(unique_values)
      print("Unique values sorted, head and tail:")
      show_head_and_tail(sorted)
  except TypeError as error:
      print(f"Could not sort values: {error}")
      print("..so let's try filtering NULL values and then sorting")
      non_null_uniques = df.loc[~df[column_name].isnull(), column_name].unique()
      sorted = np.sort(non_null_uniques)
      show_head_and_tail(sorted)


def cast_to_type(column, maybe_type):
  try:
    column.astype(maybe_type)
    print(f"Casting to {maybe_type} was successful")
  except ValueError as error:
    print(f"Could not cast to {maybe_type}: {error}")


def find_non_default_missing_values(df, column_name, maybe_type):
  long_separator_amount = 80
  short_separator_amount = 40

  print_separator("*", long_separator_amount, f"Finding non default missing values for column \"{column_name}\"")

  print(f"Column \"{column_name}\" has datatype: {df.dtypes[column_name]}")

  column = df[column_name]  

  # A
  print_separator("-", short_separator_amount, "A: Looking at unique values")
  look_at_unique_values(column)

  # B
  print_separator("-", short_separator_amount, "B: Sorting and looking at the edges")
  look_at_edges(df, column_name)

  # C
  print_separator("-", short_separator_amount, f"C: Casting to type: {maybe_type}")
  cast_to_type(column, maybe_type)

  # D
  print_separator("-", short_separator_amount, "D: Looking at frequency")
  display(column.value_counts(dropna=False))

  print("\n")

def replace_value(df, column_name, missing_old, missing_new):
    df[column_name] = df[column_name].replace({missing_old: missing_new})

In [ ]:
# Drop all rows where there is not any value

df3.dropna(axis = 0, how = 'all', inplace = True)

# After analysis, chance values that are not right with None.

replace_value(df3, "Species","Shark involvement prior to death was not confirmed", None)
replace_value(df3, "Species","Invalid", None)
replace_value(df3, "Species","Shark involvement not confirmed", None)
replace_value(df3, "Species","Shark involvement prior to death unconfirmed", None)
replace_value(df3, "Species",np.nan , None)

df3["Species"] = df3["Species"].transform(lambda x: x.lower() if x is not None else x)
df3["Species"] = df3["Species"].astype("string")

list_of_common_sharks = ["white", "tiger", "bull", "wobbegong", "blacktip", "blue", "mako", "bronze whaler", "nurse", "hammerhead", "raggedtooth", "zambesi", "shark"]

for shark in list_of_common_sharks:  
      df3.loc[df3["Species"].str.contains(shark), "Species"] = shark

most_common_sharks = df3["Species"].value_counts().head(12).astype("string")
most_common = df3["Species"].value_counts().keys()[:12]

# find_non_default_missing_values(df3, "Species", "string")

replace_value(df3, "Fatal (Y/N)","UNKNOWN", np.nan)
replace_value(df3, "Fatal (Y/N)"," N", "N")
replace_value(df3, "Fatal (Y/N)","N ", "N")
replace_value(df3, "Fatal (Y/N)","y", "Y")
replace_value(df3, "Fatal (Y/N)","2017", np.nan)
replace_value(df3, "Fatal (Y/N)","M", np.nan)

#find_non_default_missing_values(df3, "Fatal (Y/N)", "string")

display(df3)


In [ ]:
df_sharks = pd.DataFrame(most_common_sharks,columns=["Species", 'Total'])
df_sharks.reset_index(inplace=True)
df_sharks.rename(columns={"index": "Species", "Species": "Total", "Total": "Delete"}, inplace=True)
df_sharks.drop(columns= "Delete", inplace=True)
df4 = df3[df3['Species'].isin(most_common)]
df5 = df4.groupby(by="Species")["Fatal (Y/N)"].value_counts().reset_index(name='counts')

df6 = pd.merge(df5, df_sharks, on="Species")
df6["Total"] = df6["Total"].astype("int64")

df6["Average"] = (df6["counts"] / df6["Total"] *100).round(2)

df7 = df6.loc[df6['Fatal (Y/N)'] == "Y"]
df7.sort_values(by="Average", ascending=False, inplace=True)
display(df7)

In [ ]:
# 2 Are children more likely to be attacked by sharks?
# 158 different values, datatype object

df3.dropna(axis = 0, how = 'all', inplace = True)
display(df3.shape)

df3["Age"] = df3["Age"].transform(lambda x: x.lower() if x is not np.nan else x)
replace_value(df3, "Age","teen", "10")
replace_value(df3, "Age","20s", "20")
replace_value(df3, "Age","30s", "30")

all_ages = df3["Age"].value_counts()
df_all_ages = pd.DataFrame(all_ages ,columns=["Age", 'Total'])
df_all_ages.reset_index(inplace=True)
df_all_ages.rename(columns={"index": "Age", "Age": "Total", "Total": "Delete"}, inplace=True)
df_all_ages.drop(columns= "Delete", inplace=True)

df_ages_new = pd.merge(df3, df_all_ages, on="Age")
df8 = df_ages_new.loc[df_ages_new["Total"] > 3]

df8["Age"] = df8["Age"].astype("int64")

def mapping(rating):
    if rating <= 18:
        return "child"
    else:
        return "adult"
    
df8["Age"] = df8["Age"].map(mapping)
  
child_adult = df8["Age"].value_counts()
display(child_adult)



# find_non_default_missing_values(df8, "Age", "int64")

In [ ]:
# 3 Are shark attacks where sharks were provoked more or less dangerous?

replace_value(df3, "Type","Sea Disaster", "Unprovoked")
replace_value(df3, "Type","Boat", np.nan)
replace_value(df3, "Type","Boating", np.nan)
replace_value(df3, "Type","Boatomg", np.nan)
replace_value(df3, "Type","Questionable", "Invalid")

df_provoke = df3.dropna(axis = 0, subset=['Type'])

new = df_provoke.groupby(['Type','Fatal (Y/N)'])['Type'].count()
display(new)

# find_non_default_missing_values(df_provoke, "Type", "int")
# find_non_default_missing_values(df_provoke, "Activity", "string")

Type        Fatal (Y/N)
Invalid     N                16
            Y                 8
Provoked    N               548
            Y                19
Unprovoked  N              3417
            Y              1349
Name: Type, dtype: int64

In [ ]:
# 4 Are certain activities more likely to result in a shark attack?

df_acti = df3
df_acti.dropna(axis = 0, how = 'all', inplace = True)
list_of_common_activities = ["surf", "fish", "diving"]

df_acti["Activity"] = df_acti["Activity"].transform(lambda x: x.lower() if x is not np.nan else x)

df_acti["Activity"] = df_acti["Activity"].astype("string")
df_acti.loc[df_acti["Activity"].str.contains("surf"), "Activity"] = "Surfing"
df_acti.loc[df_acti["Activity"].str.contains("swim"), "Activity"] = "Swimming"
df_acti.loc[df_acti["Activity"].str.contains("fish"), "Activity"] = "Fishing"
df_acti.loc[df_acti["Activity"].str.contains("diving"), "Activity"] = "Diving"

activity_df = df_acti["Activity"].value_counts().head(50).astype("string")

display(activity_df)

# find_non_default_missing_values(df3, "Activity", "string")